In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
input_dim = 784
hidden_dim = 400
z_dim = 20
epochs = 15
batch_size = 80
learning_rate = 1e-3

# MNIST dataset
dataset = torchvision.datasets.MNIST(root='data',
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mean = nn.Linear(hidden_dim, z_dim)
        self.fc_std = nn.Linear(hidden_dim, z_dim)
        self.fc2 = nn.Linear(z_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, input_dim)
        
    def encoder(self, x):
        encode_out = F.relu(self.fc1(x))
        mean = self.fc_mean(encode_out)
        std = self.fc_std(encode_out)
        return mean, std
    
    def reparameterize(self, mean, log_var):
        std = torch.exp(log_var/2)
        sample = torch.randn_like(std)
        return mean + sample  * std

    def decoder(self, z):
        out_decode = F.relu(self.fc2(z))
        return F.sigmoid(self.fc3(out_decode))
    
    def forward(self, x):
        mean, log_var = self.encoder(x)
        z = self.reparameterize(mean, log_var)
        reconsted = self.decoder(z)
        return reconsted, mean, log_var

In [ ]:
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Start training
for epoch in range(epochs):
    for i, (images, _) in enumerate(data_loader):
        # Forward pass
        images = images.to(device).view(-1, input_dim)
        reconstructed, mean, log_var = model(images)
        
        # Compute reconstruction loss and kl divergence
        # For KL divergence, see Appendix B in VAE paper 
        reconst_loss = F.binary_cross_entropy(reconstructed, images, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
        
        # Backprop and optimize
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print ("Epoch[{}/{}], Reconstruction Loss: {:.4f}, KL Div Loss: {:.4f}" .format(epoch+1, epochs, reconst_loss.item(), kl_div.item()))
            
    with torch.no_grad():
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decoder(z).view(-1, 1, 28, 28)
        save_image(out, 'sampled-{}.png'.format(epoch+1))